In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [2]:
labels = {
    0: "buildings",
    1: "forest",
    2: "glacier",
    3: "mountain",
    4: "sea",
    5: "street"
}

In [3]:
# train_path = "../input/ml-olympiad-landscape-image-classification/train"

In [4]:
# train_datagen = ImageDataGenerator(
#         rescale = 1/255.0,
#         rotation_range=20,
#         zoom_range=0.05,
#         width_shift_range=0.05,
#         height_shift_range=0.05,
#         shear_range=0.05,
#         horizontal_flip=True,
#         fill_mode="nearest",
#         validation_split=0.20)

# # test_datagen = ImageDataGenerator(rescale = 1/255.0)

In [5]:
# train_generator = train_datagen.flow_from_directory(
#     directory=train_path,
#     target_size=(256, 256),
#     color_mode="rgb",
#     batch_size=32,
#     class_mode="categorical",
#     subset='training',
#     shuffle=True,
#     seed=42
# )

# val_generator = train_datagen.flow_from_directory(
#     directory=train_path,
#     target_size=(100, 100),
#     color_mode="rgb",
#     batch_size=32,
#     class_mode="categorical",
#     subset='validation',
#     shuffle=True,
#     seed=42
# )

In [6]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='../input/ml-olympiad-landscape-image-classification/train',
    labels='inferred',
    label_mode='categorical',
    batch_size=64,
    image_size=(256, 256),
    validation_split=0.2,
    subset="training",
    seed=1024
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='../input/ml-olympiad-landscape-image-classification/train',
    labels='inferred',
    label_mode='categorical',
    batch_size=64,
    image_size=(256, 256),
    validation_split=0.2,
    subset="validation",
    seed=1024
)

Found 14034 files belonging to 6 classes.
Using 11228 files for training.


2022-08-04 05:30:28.679431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-04 05:30:28.810523: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-04 05:30:28.811326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-04 05:30:28.814059: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Found 14034 files belonging to 6 classes.
Using 2806 files for validation.


In [7]:
from tensorflow.keras.applications import EfficientNetB7

In [8]:
base_model = EfficientNetB7(
    input_shape=(256,256,3),
    include_top=False,
    weights="imagenet")

258088960/258076736 [==============================] - 3s 0us/step


In [9]:
base_model.trainable = True

In [10]:
for layer in base_model.layers[:400]:
  layer.trainable =  False

In [11]:
def create_model():
    model = tf.keras.Sequential([base_model,
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation="elu"),
    layers.Dense(32, activation="elu"),
    layers.Dense(16, activation="elu"),
    layers.Dense(6, activation="softmax")], name='convnet')
    
    return model

In [12]:
model = create_model()

optimizer = tf.keras.optimizers.Adam()

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001/10),loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)  
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=(0.0001/10)/100000)


In [15]:
model.summary()

Model: "convnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb7 (Functional)  (None, 8, 8, 2560)        64097687  
_________________________________________________________________
random_flip (RandomFlip)     (None, 8, 8, 2560)        0         
_________________________________________________________________
random_rotation (RandomRotat (None, 8, 8, 2560)        0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                163904    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528 

In [16]:
model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=[es, reduce_lr], verbose=1)

Epoch 1/10


2022-08-04 05:31:02.585166: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-08-04 05:31:10.388306: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
2022-08-04 05:31:16.856882: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.62GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-08-04 05:31:16.888490: I tensorflow/core/util/cuda_solvers.cc:180] Creating CudaSolver handles for stream 0x55fb677e5480
2022-08-04 05:31:19.026408: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.57GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-

175/176 [============================>.] - ETA: 1s - loss: 1.3161 - accuracy: 0.5669

2022-08-04 05:34:30.283909: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.32GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-08-04 05:34:30.367251: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.58GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-08-04 05:34:30.384360: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.55GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-08-04 05:34:30.391251: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Alloc

176/176 [==============================] - 256s 1s/step - loss: 1.3152 - accuracy: 0.5675 - val_loss: 0.9562 - val_accuracy: 0.7448
Epoch 2/10
176/176 [==============================] - 218s 1s/step - loss: 0.6929 - accuracy: 0.8269 - val_loss: 0.5461 - val_accuracy: 0.8553
Epoch 3/10
176/176 [==============================] - 211s 1s/step - loss: 0.4460 - accuracy: 0.8814 - val_loss: 0.4188 - val_accuracy: 0.8810
Epoch 4/10
176/176 [==============================] - 211s 1s/step - loss: 0.3309 - accuracy: 0.9079 - val_loss: 0.3676 - val_accuracy: 0.8913
Epoch 5/10
176/176 [==============================] - 209s 1s/step - loss: 0.2656 - accuracy: 0.9247 - val_loss: 0.3426 - val_accuracy: 0.8956
Epoch 6/10
176/176 [==============================] - 211s 1s/step - loss: 0.2124 - accuracy: 0.9380 - val_loss: 0.3261 - val_accuracy: 0.8991
Epoch 7/10
176/176 [==============================] - 212s 1s/step - loss: 0.1705 - accuracy: 0.9561 - val_loss: 0.3211 - val_accuracy: 0.9027
Epoch 8/10

In [17]:
test_image_names = os.listdir('../input/ml-olympiad-landscape-image-classification/test')

label = []

for image_path in test_image_names:
    image_array = tf.keras.preprocessing.image.img_to_array(
                      tf.keras.preprocessing.image.load_img(
                          os.path.join('../input/ml-olympiad-landscape-image-classification/test', image_path)
                            )
    )
    image_array_with_batchdim = image_array[np.newaxis, :]
    
    model_prediction = np.argmax(model.predict(image_array_with_batchdim), axis=-1)[0]
    
    label.append(labels[model_prediction])

In [18]:
df = pd.DataFrame(data={
    "image": test_image_names,
    "label": label
})

df.to_csv("submission.csv", index=False)